In [2]:
# import all the relevant libraries
import wfdb
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, hamming_loss
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.preprocessing import FunctionTransformer,StandardScaler
from mne.datasets import sample
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from mne_features.feature_extraction import extract_features
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest,f_classif
import neurokit2 as nk
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE

c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods.

c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\piotr\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Return

c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to comput

c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Returning empty vector.
  warn(
c:\Users\piotr\anaconda3\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\piotr\anaconda3\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
c:\Users\piotr\anaconda3\Lib\site-packages\neurokit2\signal\signal_period.py:84: NeuroKitWarning: Too few peaks detected to compute the rate. Return

In [30]:
record_path=fr"C:\Users\piotr\Desktop\PSG data\MIT Data\slp01a"
annotation = wfdb.rdann(record_path, 'st')

# Get the sampling frequency from the record
sampling_rate = wfdb.rdrecord(record_path).fs  

# Convert annotation sample indices to times in seconds
annotation_times = np.array(annotation.sample) / sampling_rate  

# Extract annotation labels and relable 0 for no apnea and 1 for apnea
annotation_labels = np.where(np.array(annotation.symbol)=="N",0, 1).astype(np.int32)
# get the signal
signal,dic = wfdb.rdsamp(record_path)
signal=signal.flatten()
annotation_times = annotation_times[1:-1]  
annotation_labels = annotation_labels[1:-1]


In [33]:
print(sampling_rate)

250


In [37]:
#(time,label)
def relabel(label):
    if "H" in label or "HA" in label:  # Hypopnea
            return 1
    elif " A" in label or "X" in label:  # Obstructive Apnea
        return 1
    elif "CA" in label or "CAA" in label:  # Central Apnea
        return 1
    else:  # Other
        return 0
labels=[]
for i, note in enumerate(annotation.aux_note):
    labels.append((annotation.sample[i],relabel(note)))

In [ ]:


df=pd.DataFrame()
bad_times=[]
for i in range(len(annotation_times)):
    try:
        # Clean the ECG signal
        time=int(annotation_times[i])*sampling_rate
        clean = nk.ecg_clean(signal[time-3000:time+3000], sampling_rate=sampling_rate)
        
        # Compute ECG quality and filter bad segments
        quality = np.mean(nk.ecg_quality(clean, sampling_rate=sampling_rate))
        
        if quality < 0.5:
            bad_times.append(i)  # Mark as bad quality
            print(f"Warning: ECG quality check failed at index {i}. Marking as bad data. Error: {e}")
        else:
            # Extract features from cleaned ECG
            peaks = nk.ecg_peaks(clean, sampling_rate=sampling_rate)
            features = nk.hrv(peaks[0], sampling_rate=sampling_rate)  
            df = pd.concat([df, features])

    except Exception as e:
        # If any error occurs, consider this segment as bad
        bad_times.append(i)
        print(f"Warning: ECG quality check failed at index {i}. Marking as bad data. Error: {e}")
features= df.to_numpy()
labels=np.delete(annotation_labels,bad_times)